# Topic Modeling Song Lyrics

We will perform topic modeling using two techniques: Latent Dirichlet Allocation (LDA) and Non-negative Matrix Factorization (NMF) using tools from scikit-learn and gensim. All topic modeling code is contained in the `topic_modeling.py` script.

In [24]:
import pandas as pd
import numpy as np

In [25]:
data = pd.read_csv('data/lyrics_bob-dylan.csv')
data = data.dropna()

## Perform TFIDF Vectorization

Before we can start topic modelling, we must apply term frequency-inverse document frequency (TFIDF) vectorization to our tokenized dataset. TFIDF is used to determine how important a word is to a document in a collection or corpus ([ref](https://www.wikiwand.com/en/Tf%E2%80%93idf)). For example, let's say the word "like" is very popular across all songs. Using TFIDF, we downweight the importance of "like" because it is a word that occurs frequently within our corpus. Let's say "democracy" is another word within that song but it is very rare across all songs. Its importance would be upweighted using TFDIF because it doesn't occur very often in our corpus.

Note: scikit-learn's `TfidfVectorizer` expects an array of strings. So, we will need to concatenate our tokenized words together as a string for TFIDF to work properly. That being said, our concatenated tokenized words are very different from our original lyrics because we filtered out stopwords and performed lemmatization.

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', min_df=3, max_df=0.9)
X = tfidf.fit_transform(data['processed_lyrics'])
print("TFIDF matrix dimensions:",X.shape)

TFIDF matrix dimensions: (596, 2621)


In [62]:
X

<596x2621 sparse matrix of type '<class 'numpy.float64'>'
	with 34582 stored elements in Compressed Sparse Row format>

Now that we have our TFIDF matrix, we can start topic modeling with NMF and LDA.

## Non-negative Matrix Factorization (NMF)

NMF starts with a Document-Word Matrix, $DWM_{ij}$, which represents the number of occurences of word $w_j$ in document $d_i$. We create our DWM using tf-idf or count vectorization. This matrix gets factorized into two smaller matrices: Word-Topic Matrix $WTM$ and Topic-Document Matrix $TDM$.

In [63]:
from sklearn.decomposition import NMF

In [66]:
k_topics = 8
nmf = NMF(n_components=k_topics)
nmf.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=8, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [67]:
tfidf_features = tfidf.get_feature_names()
top_n_words = 20

for i in range(0,k_topics):
    topic = pd.DataFrame(data={'word':tfidf_features, 'weight':nmf.components_[i]})
    sorted_topic = topic.sort_values('weight', ascending=False).head(top_n_words)
    print("Topic %s:" % (i+1), ' '.join(sorted_topic['word']))

Topic 1: time say like know day come got long gone night heart eye tell man way hard good think away old
Topic 2: instrumental babe lookin gather stone fall rain shine hey sky mornin awake light twice wait use walkin floor everybody ledge
Topic 3: baby want got right come like mind honey night know wrong babe man unto stay really hurt lookin blue lover
Topic 4: love let true know heart want make pure blue like world need seen tonight girl fool little anybody moon hold
Topic 5: lord die home plow believe ground fixin day child hand worry hold yes jackson highway whistle lucky gone engine george
Topic 6: gon mama let high trouble levee make know goin water best easy road wake yes tell walk friend way come
Topic 7: knock heaven knockin door close trying let anymore walking like leave miss lucky orleans sister hurry baltimore wading missouri new
Topic 8: said went man got asked took right little clothes dream war shelter started highway heard come god kinda seen came


## Latent Dirichlet Allocation (LDA)

In [70]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

k_topics = 8
lda = LDA(n_components=k_topics, max_iter=10, learning_method='online')
lda.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=8, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [71]:
tfidf_features = tfidf.get_feature_names()
top_n_words = 20
for i in range(0,k_topics):
    topic = pd.DataFrame(data={'word':tfidf_features, 'weight':lda.components_[i]})
    sorted_topic = topic.sort_values('weight', ascending=False).head(top_n_words)
    print("Topic %s:" % i, ' '.join(sorted_topic['word']))

Topic 0: guillotine unlike mouthpiece disillusioned limited scuff plier stuffed obscenity baptized handcuff remark honesty meantime despise unclear lodged outsider reappear sex
Topic 1: knock knockin heaven stoned stone door congressman prophesize critic rapidly writer stalled rattle drenched senator breakfast admit libido ragin mortician
Topic 2: ramble wan dawn break happens baby gon told day know say got quit old killed night babe played like line
Topic 3: know got love like come baby time gon said say want man night let day tell heart gone way away
Topic 4: jane blowing land fair babe come whistle maid queen beauty gypsy head gon summer like bright young true nothin little
Topic 5: father seven billy broken glory good ship night drank mama danced dream sun path day wind tree best hour goin
Topic 6: instrumental mood said water come let high lord risin gon low got child turn need trouble mind day hear little
Topic 7: success failure speaks madam dangles ideal matchstick horseman quo

In [72]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, X, tfidf, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics          x          y
topic                                                  
3      86.093000        1       1  14.272788  -1.598282
7       2.160377        1       2  20.918560 -46.289513
0       2.105977        1       3  53.364021 -24.271191
1       2.105795        1       4  56.366192  14.823011
6       2.003240        1       5 -30.773230   1.871839
2       1.891574        1       6 -16.531700 -34.660961
5       1.820019        1       7 -11.104020  35.793705
4       1.820018        1       8  27.680124  41.549057, topic_info=     Category      Freq          Term      Total  loglift  logprob
term                                                              
1154  Default  2.000000  instrumental   2.000000  30.0000  30.0000
1054  Default  4.000000        heaven   4.000000  29.0000  29.0000
621   Default  6.000000          door   6.000000  28.0000  28.0000
2187  Default  3.000000         stone   3.000000  27.0000  27.0000
1234  Default  1.000000         knock   1.000000  26.0000  26.0000
231   Default  2.000000         break   2.000000  25.0000  25.0000
544   Default  2.000000          dawn   2.000000  24.0000  24.0000
2481  Default  1.000000           wan   1.000000  23.0000  23.0000
2555  Default  5.000000          wind   5.000000  22.0000  22.0000
1387  Default  4.000000          mama   4.000000  21.0000  21.0000
1305  Default  5.000000         light   5.000000  20.0000  20.0000
1626  Default  5.000000        people   5.000000  19.0000  19.0000
772   Default  6.000000          feel   6.000000  18.0000  18.0000
1039  Default  5.000000          head   5.000000  17.0000  17.0000
940   Default  5.000000           god   5.000000  16.0000  16.0000
1509  Default  6.000000          need   6.000000  15.0000  15.0000
1071  Default  5.000000          high   5.000000  14.0000  14.0000
2241  Default  5.000000           sun   5.000000  13.0000  13.0000
1091  Default  6.000000          home   6.000000  12.0000  12.0000
2530  Default  6.000000          went   6.000000  11.0000  11.0000
1824  Default  5.000000      remember   5.000000  10.0000  10.0000
2572  Default  6.000000         woman   6.000000   9.0000   9.0000
2371  Default  6.000000          town   6.000000   8.0000   8.0000
547   Default  5.000000          dead   5.000000   7.0000   7.0000
1300  Default  6.000000          life   6.000000   6.0000   6.0000
2206  Default  4.000000        street   4.000000   5.0000   5.0000
1023  Default  6.000000          hard   6.000000   4.0000   4.0000
719   Default  5.000000     everybody   5.000000   3.0000   3.0000
1517  Default  5.000000           new   5.000000   2.0000   2.0000
1860  Default  6.000000          road   6.000000   1.0000   1.0000
...       ...       ...           ...        ...      ...      ...
847    Topic8  0.024010        forest   0.666965   0.6821  -7.8541
2215   Topic8  0.023926     strolling   0.558883   0.8553  -7.8576
2596   Topic8  0.023819         wreck   0.450791   1.0658  -7.8621
1252   Topic8  0.024671          land   3.819374  -1.0359  -7.8270
1379   Topic8  0.024352          maid   1.368658  -0.0226  -7.8400
2542   Topic8  0.024354       whistle   2.086386  -0.4442  -7.8399
1755   Topic8  0.024331         queen   2.230458  -0.5119  -7.8408
2393   Topic8  0.024063         trial   0.952222   0.3282  -7.8519
2024   Topic8  0.024059         shout   0.945561   0.3350  -7.8521
1380   Topic8  0.024006        maiden   0.786133   0.5175  -7.8543
1854   Topic8  0.023928           rip   0.579578   0.8191  -7.8575
80     Topic8  0.024524          babe   6.919725  -1.6361  -7.8329
122    Topic8  0.024188        beauty   1.762732  -0.2824  -7.8467
1885   Topic8  0.024008       rounder   0.879298   0.4056  -7.8542
411    Topic8  0.024513          come  15.797404  -2.4621  -7.8334
999    Topic8  0.024169         gypsy   2.117136  -0.4664  -7.8475
1039   Topic8  0.024138          head   5.690311  -1.4564  -7.8488
947    Topic8  0.024133           gon  13.56687

## Comparing NMF vs. LDA

Latent Dirichlet Allocation (LDA) and Non-negative Matrix Factorization (NMF) are both topic modelling tools. The main difference is that LDA takes a Bayesian approach and adds a Dirichlet prior on top of the generative model. NMF’s topic-word probability distributions are fixed, while LDA’s topic-word distributions vary based on how the prior was tuned (hyperparameter $k$ - number of components). NMF would be a better choice if the topic probabilities are fixed for each document ref. Also, if our dataset is small, LDA may have inferior performance since it could introduce too much variability to the model ref.

Unlike NMF, reconstructing X with LDA is not a closed-form solution. We need to use Monte Carlo simulations to sample from the distribution of Z (the distribution of topics for each sample), followed by the distribution of W (the distribution of words for topic $Z_i$).